<a href="https://colab.research.google.com/github/CPallaresU/TALLERES-CEIA/blob/main/SEMANA_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

##EJERCICIO #1

In [ ]:
def normalizacion(dataset,cols,rows):
  filas=rows
  columnas=cols
  data=dataset
  mu=np.mean(data,axis=0)
  std = np.std(data,axis=0)
  data=(data-mu)/std
  data = data.reshape(rows,cols)
  return data

##EJERCICIO #2

In [ ]:
def remove_NaN(X,axis_):
  if axis_==1: #Filas
    t=~np.isnan(X).any(axis=axis_)
    #X=X.T
    X=X[t]
    #X=X.T
  else: #Columnas
    t=~np.isnan(X).any(axis=axis_)
    X=X[t]
  
  return X

##EJERCICIO #3

In [ ]:
def replace_NaN(data): #Código de https://stackoverflow.com/questions/18689235/numpy-array-replace-nan-values-with-average-of-columns
  col_mean = np.nanmean(a, axis=0) #Calculamos la media de cada column
  inds = np.where(np.isnan(a)) # Obtenemos las filas donde la columna es NaN
  a[inds] = np.take(col_mean, inds[1]) #remplazamos los valores de NaN por la media de su respectiva columna
                                       #y las coordenadas de NaN están en inds

##EJERCICIO #4

In [ ]:
def split_data(data):

  data.sample(frac=1)
  x,y=np.shape(data)

  print(x)
  print(y)

  train=data.iloc[:int(x*.7)]
  validation=data.iloc[int(x*.7):int(x*.9)]
  test=data.iloc[int(x*.9):]

  return np.array(train),np.array(validation),np.array(test)

##Integrador

In [ ]:
q=pd.read_csv("income.csv")
w=remove_NaN(np.array(q),1)
df = pd.DataFrame(w, columns = q.columns)
train,validation,test=split_data(df)

498
3


##PCA

In [ ]:
n=2 
pca = PCA(n_components=n)
new_img_pca=pca.fit_transform(train)
pca.transform(validation)

array([[ 1.74498283e+02,  1.58793832e+00],
       [ 1.75502034e+02, -1.97827824e+00],
       [ 1.76501909e+02, -1.63166245e+00],
       [ 1.77500702e+02, -5.09247238e-01],
       [ 1.78501425e+02, -1.40569670e+00],
       [ 1.79499914e+02,  2.69649595e-01],
       [ 1.80503808e+02, -3.76295018e+00],
       [ 1.81502285e+02, -2.28640881e+00],
       [ 1.82502831e+02, -2.49202443e+00],
       [ 1.83496394e+02,  3.45567865e+00],
       [ 1.84502051e+02, -1.70469376e+00],
       [ 1.85503428e+02, -2.97338839e+00],
       [ 1.86499509e+02,  8.02935479e-01],
       [ 1.87500517e+02, -3.36987345e-01],
       [ 1.88501181e+02, -1.33491315e+00],
       [ 1.89497250e+02,  2.73358513e+00],
       [ 1.90502881e+02, -2.30457121e+00],
       [ 1.91503449e+02, -3.11431916e+00],
       [ 1.92497363e+02,  2.71814130e+00],
       [ 1.93497677e+02,  1.92879776e+00],
       [ 1.94497647e+02,  1.96046445e+00],
       [ 1.95501411e+02, -1.15422027e+00],
       [ 1.96501845e+02, -1.58247057e+00],
       [ 1.

##Regresión lineal

In [ ]:
class model_class(object):

  def __init__(self):
      self.model = None

  def fit(self, X, Y):
      return NotImplemented

  def predict(self, X):
      return NotImplemented

class linear_Reg (model_class):

  def fit(self,x,y):
     w = np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y) #utilizamos esta formula a partir de despejar el parametro W del error cuadratico medio 
     self.weight=w                                 #igualado a cero y derivado, obteniendo el optimo W
     return self.weight

  def predict(self,x,parameters):
    
    y=np.dot(x,parameters)
    return y

In [107]:
class metric(object):

  def __init__(self,y_actual,y_pred,TP,FP,TN,FN):
    self.y_actual=y_actual
    self.y_pred=y_pred
    self.TP = TP
    self.FP = FP
    self.TN = TN
    self.FN = FN

  def vals_(self):

    for i in range(len(self.y_pred)): 
        if self.y_actual[i]==self.y_pred[i]==1:
           self.TP += 1
        if self.y_pred[i]==1 and self.y_actual[i]!=self.y_pred[i]:
           self.FP += 1
        if self.y_actual[i]==self.y_pred[i]==0:
           self.TN += 1
        if self.y_pred[i]==0 and self.y_actual[i]!=self.y_pred[i]:
           self.FN += 1
    return self.TP,self.FP,self.TN,self.FN


class mse(metric):
  
  def mse_(self):
    return (sum(np.square(self.y_actual-self.y_pred))/len(self.y_pred))

class recall(metric):

  def recall_(self):
    return self.TP / (self.TP + self.FN)

class precision(metric):

  def precision_(self):
    return self.TP / (self.TP + self.FP)


In [108]:
model=linear_Reg()
parameters=model.fit(train[:,:np.shape(train)[1]-1],
          train[:,-1]) # y = x[:,np.shape(x)[1]-1] selecciona la ultima columna, el -1 es para no salir de las n-1 dimensiones

y_pred=model.predict(test[:,:np.shape(test)[1]-1],parameters)

In [110]:
error = mse(test[:,-1],y_pred,0,0,0,0)
print("Error asociado a la predicción: {}".format(mse.mse_(error)))

Error asociado a la predicción: 0.5480442786289866
